# Prepare Input Data
We are going to train an MPNN to predict the B3LYP-level atomization energy.
The first step will be to convert that data to tensorflow records

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import json

RDKit WARNING: [08:53:28] Enabling RDKit 2019.09.3 jupyter extensions


## Get the Data
It is stored on a [GitHub page](https://github.com/globus-labs/g4mp2-atomization-energy) from a previous project

In [2]:
data = pd.read_json('https://github.com/globus-labs/g4mp2-atomization-energy/raw/master/data/output/g4mp2_data.json.gz', 
                   lines=True)
print(f'Downloaded {len(data)} training entries')

Downloaded 130258 training entries


## Save the Data as CSV files
We'll make both a training, validation and test set.
For simplicity, we'll just store a file with a "SMILES" and a "output" value

In [3]:
test_set = data.query('in_holdout')
print(f'Set aside {len(test_set)} training entries')

Set aside 13026 training entries


In [4]:
train_set, val_set = train_test_split(data.query('not in_holdout'), test_size=0.1, random_state=1)
print(f'Split off {len(train_set)} training and {len(val_set)} validation entries')

Split off 105508 training and 11724 validation entries


Save the data in TF format

In [5]:
out_dir = Path('data')
out_dir.mkdir(exist_ok=True)

In [8]:
for name, dataset in zip(['train', 'valid', 'test'], [train_set, val_set, test_set]):
    dataset = dataset.sample(frac=1.)  # Shuffle contents
    dataset.rename(columns={'smiles_0': 'smiles', 'g4mp2_atom': 'output'})[['smiles', 'output']].to_csv(out_dir / f'{name}.csv', index=False)